## SVM
SVM (Support Vector Machine), 支持向量积。

### SVM 简单推导

<img src="margin.png" height="200px" width="300px">

1. 分隔超平面方程

$$
f(\textbf{x}) = \textbf{w}^T\textbf{x} + b
$$

2. 点到平面的距离

$$
d = \frac{|\textbf{w}^T\textbf{x} + b|}{||\textbf{w}||}
$$


3. 目标：最大化间隔

我们的最大化间隔就是，最大化图中两个虚线中之间的最大的路径 $\frac{1}{||\textbf{w}||} + \frac{1}{||\textbf{w}||}$, 并且其它样本点的路径都是大于等于 1 的。

$$
\max_{\textbf{w}, b} \space\space\space\space \frac{2}{||\textbf{w}||}
\\
s.t. \space\space\space\space y_i (\textbf{w}^T \textbf{x}_i + b) \geqslant 1, \space\space i = 1,..., m
$$

4. 将 3 的 max 形式取成 min 形式，即 max 的导数形式，约束条件不变。

$$
\min_{\textbf{w}, b} \space\space\space\space \frac{1}{2}{||\textbf{w}||^2}
\\
s.t. \space\space\space\space y_i (\textbf{w}^T \textbf{x}_i + b) \geqslant 1, \space\space i = 1,..., m
$$


5. 将 4 写成拉格朗日乘子式

$$
L(\textbf{w}, b, \textbf{⍺}) = \frac{1}{2}||\textbf{w}||^2 + \sum_{1}^{m}\alpha_{i}(1 - y_i(\textbf{w}^T\textbf{x}_i + b))
$$

6. 对 5 式分别求 $\frac{\partial L}{\partial \textbf{w}}, \frac{\partial L}{\partial \textbf{b}}$, 并令其等于 0, 可以得到其驻点。（**为什么凸优化，因为求得的驻点就是极值点**）。

$$
\textbf{w} = \sum_{1}^{m}\alpha_i y_i \textbf{x}_i
\\
\sum_{1}^{m}\alpha_i y_i = 0
$$


7. 将驻点的解代入到 5， 就可以得到 3 的对偶问题。

$$
\max_{\textbf{⍺}} \space\space \sum_{i}^{m}\alpha_i - \frac{1}{2}\sum_{i=1}^m\sum_{j=1}^m \alpha_i \alpha_j y_i y_j K({\textbf{x}_i}, \textbf{x}_j)
\\
s.t. \space\space\space\space
\sum_{1}^{m}\alpha_i y_i = 0,  \space \alpha_i \geqslant 0
$$

其中，$K({\textbf{x}_i}, \textbf{x}_j)$ 是核。

8. 引入松驰变量（Slack variable）

如果样本允许被错分，则可以引入松驰变量 C。上面的优化目标不变，只改变下约束条件。

$$
\max_{\textbf{⍺}} \space\space \sum_{i}^{m}\alpha_i - \frac{1}{2}\sum_{i=1}^m\sum_{j=1}^m \alpha_i \alpha_j y_i y_j K({\textbf{x}_i}, \textbf{x}_j)
\\
s.t. \space\space\space\space
\sum_{1}^{m}\alpha_i y_i = 0, \space\space C \geqslant \alpha_i \geqslant 0
$$

引入松驰变量后，我们所找到间隔也称之为软间隔(soft margin)。

引入了松驰变量，也可以看成是对原目标优化的一个正则化处理。正则化可以认为是提供了模型的先验概率。

9. 求解 $\textbf{⍺}$
一旦求出 $\textbf{⍺}$, 如使用 SMO 算法求解，就可以求出 $\textbf{w}, b$ 然后我们的分隔超平面就找到了。

### 核(kernel)

从 7 中可以看到我们优化目标的核是可以任意选择的，只要核生成的**『核矩阵』是半正定（即所有特征值 $\geqslant 0$）**。 SVM 的强大功能是因为可以利用核将原始空间映射到一个更高维的特征空间上，使得样本在高维的特征空间上线性可分。

一些常用核函数：

1. 线性核

$$
K({\textbf{x}_i}, \textbf{x}_j) = {\textbf{x}_i}^T\textbf{x}_j
$$

2. 多项式核

$$
K({\textbf{x}_i}, \textbf{x}_j) = ({\textbf{x}_i}^T\textbf{x}_j)^d
$$

d = 1 时，退化成线性核。

3. 高斯核

$$
K({\textbf{x}_i}, \textbf{x}_j) = exp(-\frac{||{\textbf{x}_i} - \textbf{x}_j||^2}{2\sigma^2}) = exp(- \gamma \|{\textbf{x}_i} - \textbf{x}_j\|^2)
$$

高斯核也称为 RBF(径向基函数, Radical basis function) 核， 其中 $\sigma$ 为高斯核的带宽(width), 用于确定到达率(reach)，或者说函数值跌落到 0 的速度参数。高斯核函数将数据从特征空间映射到更高维的空间，具体的说是映射到一个无穷维空间。 一般应用 SVM 时首先 RBF 核，$\gamma$ 也成为了一个超参。

4. 拉普拉斯核

$$
K({\textbf{x}_i}, \textbf{x}_j) = exp(-\frac{||{\textbf{x}_i} - \textbf{x}_j||}{\sigma})
$$

其中 $\sigma > 0$ 。

另外核函数之间通过函数组合得到新的核函数。

### SMO
SMO(Sequential Margin Optimization) 是一种很高效求解 $\alpha_i$ 的方法。其工作原理是：
1. 每次循环选择两个 alpha 进行处理。
2. 一旦找到一对合适的 alpha, 那么就增加一个同时减小另一个。

#### SMO 算法

1. 随机初始化 $\alpha_i, b$
2. 初始化误差项，其中： $E_i = f(x_i) - y_i, f(x_i) = \sum_{j = i}^{m}\alpha_i k_{ij} + b$
3. 开始循环：
    
    a. 选择两个 $\alpha_1, \alpha_2$ 作为优化目标。然后计算 $\alpha_{2}^{new} = \alpha_{2}^{old} + \frac{y_2(E_1 - E_2)}{\eta}$
    
    b. 计算 clipped $\alpha_2^{new ,clipped}$
    
    c. 计算 $\alpha_1^{new}$
    
    d. 更新 $E_i, b$
    
    e. 如果达到终止条件则退出循环。

    